In [3]:
import pandas as pd
import numpy as np

import json
from rdkit import Chem
import pandas as pd
import itertools
import sys
from tqdm import tqdm 
from itertools import chain, combinations
import sys
import multiprocessing as mp
from functools import partial
from collections import Counter
tqdm.pandas()
sys.path.append('../')
from common import *

### input and output

In [43]:
###input
ymdb_output_final = '../../../Data/database/ymdb/ymdb_final_result.xlsx'

target_smiles_path = '../../../Results/lipid/target_smiles_lipid.pickle'
target_model_smiles_path = '../../../Results/lipid/yeast_met_lipid.pickle'
yeast8_metabolites_path = '../../../Data/yeast-GEM-final.csv'
total_met_inchikey0_file = '../../../Results/analysis/total_met_inchikey0.pickle'

cut_off = 0.3
num = 'all'
num_process = 5
cut_off_path = f'../../../Results/lipid/top{num}_{cut_off}_re/'

# cut_off_path = f'../../../Results/lipid/top{num}_{cut_off}_re_copy_new/'
sce_gene_clean_ec = '../../../Results/ECnumber_prediction/Saccharomyces_cerevisiae_teacher_maxsep.csv'
DeepEC_path = '../../../Results/ECnumber_prediction/Sce_DeepECv2.txt'
sce_uniprot_path = '../../../Data/database/uniprot/uniprotkb_organism_id_559292_2023_11_08.tsv'

yeast870_path =  '../../../Data/model/yeast-GEM.yml'
miss_met_id_file = '../../../Data/miss_met_id.json'
rxndb_to_model_with_target_success_path = f'../../../Results/lipid/top{num}_{cut_off}_re_copy_new/' + f'rxndb_to_model_with_target_top{num}_{cut_off}_new_merge.csv'
metabolites_info_to_GEM_path_not_lipid_path = '../../../Results/not_lipid/top50_0.3_re/metabolites_info_to_GEM_top50_0.3.csv'
rxndb_total_info_to_model_del_path = f'../../../Results/lipid/top{num}_{cut_off}_re/' + f'rxndb_total_info_to_model_top{num}_{cut_off}_del.csv'
metabolites_info_to_GEM_before_path = f'../../../Results/lipid/top{num}_{cut_off}_re/' + f'metabolites_info_to_GEM_top{num}_{cut_off}.csv'


###output

# rxndb_to_model_add_ec_path = cut_off_path + f'rxndb_to_model_add_ec_top{num}_{cut_off}.csv'
rxndb_GPR_to_model_path = cut_off_path + f'rxndb_GPR_to_model_top{num}_{cut_off}_remain.csv'
rxndb_met_max_score_met_annotation_file = cut_off_path + f'rxndb_met_max_score_met_annotation_top{num}_{cut_off}_remain.csv'
metabolites_info_to_GEM_path = cut_off_path + f'metabolites_info_to_GEM_top{num}_{cut_off}_remain.csv'
rxndb_total_info_to_model_path = cut_off_path + f'rxndb_total_info_to_model_top{num}_{cut_off}_remain.csv'
yeast8_reaction_in_rxndb_json = cut_off_path + f'yeast8_reaction_in_rxndb_top{num}_{cut_off}_remain.json'



In [24]:
def process_smiles_annotation(i,yeast_total_smiles):    ###The highest similarity scores of all the reactants and model metabolites of rxn were obtained
    result = {'smile': i, 'score': 0, 'sim_smile': ''}
    score_init = 0
    sim_smile = ' '
    for j in yeast_total_smiles:
        # tmp_score = compare_smiles(i, j)  
        if compare_smiles_inchikey(i,j) :
            score_init = 1
            sim_smile = j
            break
    result['score'] = score_init
    result['sim_smile'] = sim_smile
    return result
def calculate_smile_max_score_annotation(rxndb_all_reactant_smile,yeast_total_smiles,num_process=5):
    smile_max_score = {'smile': [], 'score': [], 'sim_smile': []}

    pool = multiprocessing.Pool(num_process)

    partial_process_smiles = partial(process_smiles_annotation,yeast_total_smiles=yeast_total_smiles)
    results = list(tqdm(pool.imap(partial_process_smiles, rxndb_all_reactant_smile), total=len(rxndb_all_reactant_smile)))
    # results = list(tqdm(pool.imap(process_smiles, rxndb_all_reactant_smile), total=len(rxndb_all_reactant_smile)))

    pool.close()
    pool.join()

    for result in results:
        smile_max_score['smile'].append(result['smile'])
        smile_max_score['score'].append(result['score'])
        smile_max_score['sim_smile'].append(result['sim_smile'])
        
    return(smile_max_score)

In [6]:
data = pd.read_csv(rxndb_to_model_with_target_success_path)
data.shape



(2112, 5)

In [7]:
model = cobra.io.load_yaml_model(yeast870_path)

model_genes = model.genes

model_genes = [x.id for x in model_genes]

### CLEAN

In [8]:
clean_gene2ec_dict = get_gene2ec_dict_clean(sce_gene_clean_ec)

### DeepECv2

In [9]:
DeepEC_gene2ec_dict = get_gene2ec_dict_DeepEC(DeepEC_path)

In [10]:
clean_deepec_gene2ec_dict = {}
for k,v in clean_gene2ec_dict.items():
    if k in DeepEC_gene2ec_dict or k in model_genes:
        clean_deepec_gene2ec_dict[k] = v
clean_deepec_ec2gene_dict = get_ec2gene_dict_clean(clean_deepec_gene2ec_dict)
print(len(clean_deepec_ec2gene_dict))

161


In [11]:
def rxndb_gene_annotation(rxndb,ec2gene_dict):
    rxndb = rxndb.astype(str)
    rxndb['EC number'] = rxndb['EC number'].apply(lambda x:x.split(';') if x else [])
    rxndb['EC number'] = rxndb['EC number'].apply(lambda x:list(set([".".join(value.split(".")[:3]) for value in x])))
    # rxndb = rxndb[rxndb['EC number'].apply(lambda x: len(x) > 0)]  #Eliminate rows with ec empty
    rxndb['GENE'] = rxndb['EC number'].apply(lambda x: [gene for ec in x if ec in ec2gene_dict for gene in ec2gene_dict[ec]] if x else []) 
    ####all EC
    all_EC_list = []
    for i in rxndb['EC number']:
        all_EC_list+=i
    all_EC_list = list(set(all_EC_list))
    print('rxndb all ec num',len(all_EC_list))    
    return rxndb

In [12]:
rxndb_with_GPR = rxndb_gene_annotation(data,clean_deepec_ec2gene_dict)
# rxndb_with_GPR = rxndb_with_GPR[rxndb_with_GPR['GENE'].apply(lambda x: len(x) > 0)]
# data_no_ec['GENE'] = None
# data_no_ec['GENE'] = data_no_ec['GENE'].apply(lambda x: [])
# data_no_ec['EC number'] = data_no_ec['EC number'].apply(lambda x: [])
print('rxndb with GPR:',rxndb_with_GPR.shape)


rxndb all ec num 27
rxndb with GPR: (2112, 6)


In [13]:
rxndb_with_GPR.to_csv(rxndb_GPR_to_model_path,index=None)


### rxndb metabolite annotation

In [14]:
#rxndb all metabolites
rxndb_all_smile = get_all_met_smile(rxndb_GPR_to_model_path)


100%|██████████| 2112/2112 [00:00<00:00, 19035.89it/s]

1599


In [15]:
#metabolite comparison
yeast8_total_smiles = get_GEM_all_smiles(yeast8_metabolites_path)
smile_max_score = calculate_smile_max_score_annotation(rxndb_all_smile,yeast8_total_smiles,num_process=num_process)
smile_max_score_pd = pd.DataFrame(smile_max_score)
smile_max_score_pd.to_csv(rxndb_met_max_score_met_annotation_file,index=None)


100%|██████████| 1599/1599 [01:13<00:00, 21.70it/s]


In [16]:
def assign_ID_to_RXNDB_met(smile_max_score_pd,yeast_id_smiles_mapping,s = 'lipid'):

    sim_smile_met_id = []
    smiles_id_dict = {}
    num = 13349
    for index,row in tqdm(smile_max_score_pd.iterrows(),total = smile_max_score_pd.shape[0]):
        if row['score'] == 1:
            # sim_smile_met_id.append(get_gem_met_id_for_smile(row['sim_smile'],yeast8_MNX_CHBEI_id_mapping))
            sim_smile_met_id.append(get_gem_met_id_for_smile(row['sim_smile'],yeast_id_smiles_mapping))
        else:
            inchikey0 = smiles2inchikey0(row['smile'])
            if inchikey0 in smiles_id_dict:
                id = smiles_id_dict[inchikey0]
            else:
                num +=1
                if s == 'lipid':
                    id = 'sl_' +str(num)
                else:
                    id = 'sn_' +str(num)
                smiles_id_dict[inchikey0] = id
            sim_smile_met_id.append((id,'c'))
    smile_max_score_pd['sim_smile_met_id'] = sim_smile_met_id

    return smile_max_score_pd    


In [17]:
def get_yeast8_id_smiles_mapping(model_path):
    model = pd.read_csv(model_path)
    model = model.dropna(subset=['standard_smiles'])
    yeast_id_mapping = model[['REPLACEMENT ID','COMPARTMENT','standard_smiles']]
    yeast_id_mapping.columns = ['met_id','compartment','smiles']
    yeast_id_mapping['smiles'] = yeast_id_mapping['smiles'].apply(normalize_smiles)
    yeast_id_mapping['inchikey0'] = yeast_id_mapping['smiles'].apply(smiles2inchikey0)
    return yeast_id_mapping

In [18]:
#assign ID
yeast8_id_smiles_mapping = get_yeast8_id_smiles_mapping(yeast8_metabolites_path)
smile_max_score_pd_with_ID = assign_ID_to_RXNDB_met(smile_max_score_pd,yeast8_id_smiles_mapping,s = 'lipid')
metabolites_info_to_GEM = get_new_met_smile_list(smile_max_score_pd_with_ID)
metabolites_info_to_GEM.to_csv(metabolites_info_to_GEM_path,index=None)



/tmp/ipykernel_2307632/1190804332.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yeast_id_mapping['smiles'] = yeast_id_mapping['smiles'].apply(normalize_smiles)
/tmp/ipykernel_2307632/1190804332.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yeast_id_mapping['inchikey0'] = yeast_id_mapping['smiles'].apply(smiles2inchikey0)
100%|██████████| 1599/1599 [00:02<00:00, 663.34it/s]


In [19]:
metabolites_info_to_GEM_path_not_lipid = pd.read_csv(metabolites_info_to_GEM_path_not_lipid_path)
metabolites_info_to_GEM_path_not_lipid = metabolites_info_to_GEM_path_not_lipid[~metabolites_info_to_GEM_path_not_lipid['ID'].str.contains('s_')]
metabolites_info_to_GEM_path_not_lipid['inchikey0'] = metabolites_info_to_GEM_path_not_lipid['new_met_smiles'].apply(smiles2inchikey0)
for i in tqdm(range(metabolites_info_to_GEM.shape[0]),total=metabolites_info_to_GEM.shape[0]):
    if 'sl_' not in metabolites_info_to_GEM['ID'][i]:
        continue
    if smiles2inchikey0(metabolites_info_to_GEM['new_met_smiles'][i]) in metabolites_info_to_GEM_path_not_lipid['inchikey0'].values:
        metabolites_info_to_GEM['ID'][i] = metabolites_info_to_GEM_path_not_lipid[metabolites_info_to_GEM_path_not_lipid['inchikey0'] == smiles2inchikey0(metabolites_info_to_GEM['new_met_smiles'][i])]['ID'].values[0]
metabolites_info_to_GEM.to_csv(metabolites_info_to_GEM_path,index=None)

  0%|          | 0/1599 [00:00<?, ?it/s]/tmp/ipykernel_2307632/1369954493.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metabolites_info_to_GEM['ID'][i] = metabolites_info_to_GEM_path_not_lipid[metabolites_info_to_GEM_path_not_lipid['inchikey0'] == smiles2inchikey0(metabolites_info_to_GEM['new_met_smiles'][i])]['ID'].values[0]
/tmp/ipykernel_2307632/1369954493.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metabolites_info_to_GEM['ID'][i] = metabolites_info_to_GEM_path_not_lipid[metabolites_info_to_GEM_path_not_lipid['inchikey0'] == smiles2inchikey0(metabolites_info_to_GEM['new_met_smiles'][i])]['ID'].values

In [20]:
metabolites_info_to_GEM_path_before = pd.read_csv(metabolites_info_to_GEM_before_path)
metabolites_info_to_GEM_path_before = metabolites_info_to_GEM_path_before[~metabolites_info_to_GEM_path_before['ID'].str.contains('s_')]

metabolites_info_to_GEM_path_before['inchikey0'] = metabolites_info_to_GEM_path_before['new_met_smiles'].apply(smiles2inchikey0)
for i in tqdm(range(metabolites_info_to_GEM.shape[0]),total=metabolites_info_to_GEM.shape[0]):
    if smiles2inchikey0(metabolites_info_to_GEM['new_met_smiles'][i]) in metabolites_info_to_GEM_path_before['inchikey0'].values:
        metabolites_info_to_GEM['ID'][i] = metabolites_info_to_GEM_path_before[metabolites_info_to_GEM_path_before['inchikey0'] == smiles2inchikey0(metabolites_info_to_GEM['new_met_smiles'][i])]['ID'].values[0]
metabolites_info_to_GEM.to_csv(metabolites_info_to_GEM_path,index=None)

  0%|          | 0/1599 [00:00<?, ?it/s]/tmp/ipykernel_2307632/4086319083.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metabolites_info_to_GEM['ID'][i] = metabolites_info_to_GEM_path_before[metabolites_info_to_GEM_path_before['inchikey0'] == smiles2inchikey0(metabolites_info_to_GEM['new_met_smiles'][i])]['ID'].values[0]
/tmp/ipykernel_2307632/4086319083.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metabolites_info_to_GEM['ID'][i] = metabolites_info_to_GEM_path_before[metabolites_info_to_GEM_path_before['inchikey0'] == smiles2inchikey0(metabolites_info_to_GEM['new_met_smiles'][i])]['ID'].values[0]
/tmp/ipy

### Generate equation according to metabolite ID

In [21]:
smiles_to_id_mapping = get_smiles_to_id_mapping(metabolites_info_to_GEM_path)
print(len(smiles_to_id_mapping))

1599


In [25]:
rxndb_total_info_to_model = get_rxndb_to_model_total_info(rxndb_GPR_to_model_path,smiles_to_id_mapping)
rxndb_total_info_to_model = rxndb_total_info_to_model[['NO','EC_number','templateID','rxn_smiles_basic','rxn_smiles_final','GPR','equation_dict','equation']]
print(rxndb_total_info_to_model.shape)
rxndb_total_info_to_model.to_csv(rxndb_total_info_to_model_path,index=None)


(1480, 8)


### remove duplicate reactions and existing reactions

In [26]:
same_rxndb_reaction = get_same_rxndb_reaction_mapping(rxndb_total_info_to_model_path)
rxndb_total_info_to_model = get_no_same_rxndb_reaction_parallelize(rxndb_total_info_to_model_path,get_no_same_rxndb_reaction,same_rxndb_reaction,n_cores=num_process)
print(rxndb_total_info_to_model.shape)

(1480, 8)


100%|██████████| 1480/1480 [00:00<00:00, 16547.78it/s]

(1252, 8)


In [27]:
rxndb_total_info_to_model = rxndb_total_info_to_model.drop(columns=['NO'])
rxndb_total_info_to_model = rxndb_total_info_to_model.reset_index(drop=True)
rxndb_total_info_to_model.index.name = 'NO'
rxndb_total_info_to_model = rxndb_total_info_to_model.reset_index()
rxndb_total_info_to_model['NO'] = rxndb_total_info_to_model['NO'].apply(lambda x: f'rxnl{1022175+int(x)}')
rxndb_total_info_to_model.head()

,NO,EC_number,templateID,rxn_smiles_basic,rxn_smiles_final,GPR,equation_dict,equation
0,rxnl1022175,3.1.3,MNXR114735,CCCCCCCCCCCCCC=CC(O)C([NH3+])CO.O=P([O-])([O-]...,CCCCCCCCCCCCCC=CC(O)C([NH3+])CO.O=P([O-])([O-]...,(YDR063W) or (YDR051C) or (YKR053C) or (YIL002...,"{'sn_35': -1, 's_1322': -1, 'sn_292': 1, 's_08...",sn_35 + s_1322 => sn_292 + s_0803
1,rxnl1022176,3.1.3,MNXR114735,CC(C)=CCCC(C)=CCCC(C)=CCCC(C)=CCCC(C)=CCCC(C)=...,CC(C)=CCCC(C)=CCCC(C)=CCCC(C)=CCCC(C)=CCCC(C)=...,(YDR063W) or (YDR051C) or (YKR053C) or (YIL002...,"{'sl_4995': -1, 's_1322': -1, 'sl_3211': 1, 's...",sl_4995 + s_1322 => sl_3211 + s_0803
2,rxnl1022177,2.1.1,MNXR108123_reverse,COc1c(O)c(C)c(CC=C(C)CCC=C(C)CCC=C(C)CCC=C(C)C...,COc1c(O)c(C)c(CC=C(C)CCC=C(C)CCC=C(C)CCC=C(C)C...,(YDR497C) or (YBR236C) or (YCR024C-A) or (YKL1...,"{'s_1535': -1, 's_1413': -1, 's_0794': -1, 's_...",s_1535 + s_1413 + s_0794 => s_1416 + sn_194
3,rxnl1022178,1.1.1,MNXR181637,CC(C)=CCCC(C)=CCC/C(C)=C/C=O>>C=CC(C)(O)CCC=C(...,CC(C)=CCCC(C)=CCC/C(C)=C/C=O.NC(=O)C1=CN([C@@H...,(YDR536W) or (YLR174W) or (YHR183W) or (YJL214...,"{'s_3983': -1, 's_1203': -1, 's_0794': -1, 'sn...",s_3983 + s_1203 + s_0794 => sn_143 + s_1198
4,rxnl1022179,2.4.1,MNXR147185,CC(=O)NC1C(OC2C(CO)OC(OP(=O)(O)OP(=O)(O)OCCC(C...,CC(=O)NC1C(OC2C(CO)OC(OP(=O)(O)OP(=O)(O)OCCC(C...,(YBR180W) or (YCR089W) or (YKR058W) or (YOR192...,"{'sl_5638': -1, 's_1538': -1, 's_0794': -1, 's...",sl_5638 + s_1538 + s_0794 => s_1544 + sl_...


In [44]:
rxndb_total_info_to_model = rxndb_total_info_to_model.drop_duplicates(subset=['rxn_smiles_final'])
rxndb_total_info_to_model.shape

(1062, 8)

In [45]:
rxndb_total_info_to_model.to_csv(rxndb_total_info_to_model_path,index=None)


### check the formula of the model metabolites in the predicted reaction

In [29]:
metabolites_info_to_GEM,rxndb_total_info_to_model = check_met_and_reaction_formula(yeast8_metabolites_path,metabolites_info_to_GEM_path,rxndb_total_info_to_model_path)
metabolites_info_to_GEM.to_csv(metabolites_info_to_GEM_path,index=None)
rxndb_total_info_to_model.to_csv(rxndb_total_info_to_model_path,index=None)
print(rxndb_total_info_to_model.shape)

formula error_met set()


0it [00:00, ?it/s]

(1252, 8)


In [30]:
met_info = pd.read_csv(metabolites_info_to_GEM_path)
rxndb_to_model_total_info = pd.read_csv(rxndb_total_info_to_model_path)


### Find out the reactions that already exist in the model

In [31]:
with open(miss_met_id_file) as f:
    miss_met_id = json.load(f) 
# model = cobra.io.load_yaml_model(yeast870_path)


### yeast8_smiles_list, ID with smiles

In [32]:
model_excel, metID_with_smiles, yeast8_smiles_lst = process_model(yeast8_metabolites_path, miss_met_id)


2124
['s_0006', 's_0007', 's_0008', 's_0009', 's_0010', 's_0011', 's_0012', 's_0013', 's_0014', 's_0015', 's_0016', 's_0017', 's_0018', 's_0019', 's_0020', 's_0021', 's_0022', 's_0023', 's_0024', 's_0025', 's_0026', 's_0027', 's_0028', 's_0029', 's_0030', 's_0031', 's_0032', 's_0033', 's_0034', 's_0035', 's_0036', 's_0037', 's_0038', 's_0039', 's_0042', 's_0045', 's_0048', 's_0051', 's_0054', 's_0056', 's_0058', 's_0060', 's_0061', 's_0062', 's_0063', 's_0064', 's_0065', 's_0066', 's_0067', 's_0068', 's_0069', 's_0075', 's_0076', 's_0077', 's_0078', 's_0079', 's_0080', 's_0086', 's_0087', 's_0116', 's_0117', 's_0118', 's_0119', 's_0120', 's_0121', 's_0122', 's_0123', 's_0124', 's_0125', 's_0126', 's_0127', 's_0129', 's_0131', 's_0132', 's_0133', 's_0134', 's_0135', 's_0136', 's_0137', 's_0138', 's_0139', 's_0140', 's_0141', 's_0142', 's_0143', 's_0144', 's_0145', 's_0146', 's_0147', 's_0148', 's_0149', 's_0150', 's_0152', 's_0153', 's_0154', 's_0155', 's_0156', 's_0157', 's_0158', 's_0

In [33]:
rxndb_met_with_s_smiles_lst = get_rxndb_met_with_s_smiles_lst(rxndb_total_info_to_model_path,model_excel,miss_met_id)

reaID_with_smiles_dict = get_reaID_with_smiles_dict(model, model_excel, miss_met_id,rxndb_met_with_s_smiles_lst,metID_with_smiles)


(1252, 8)


100%|██████████| 1252/1252 [00:00<00:00, 2338.98it/s]


104
['COc1c(O)c(C)c(CC=C(C)CCC=C(C)CCC=C(C)CCC=C(C)CCC=C(C)CCC=C(C)C)c(O)c1OC', 'Nc1ncnc2c1ncn2C1OC(CSCCC(N)C(=O)O)C(O)C1O', 'C[S+](CCC(N)C(=O)[O-])CC1OC(n2cnc3c(N)ncnc32)C(O)C1O', 'CC(C)=CCCC(C)=CCCC(C)=CC=O', 'O=c1ccn(C2OC(COP(=O)(O)OP(=O)(O)O)C(O)C2O)c(=O)[nH]1', 'CC(=O)NC1C(OP(=O)(O)OP(=O)(O)OCC2OC(n3ccc(=O)[nH]c3=O)C(O)C2O)OC(CO)C(O)C1O', 'O=c1ccn(C2OC(COP(=O)(O)OP(=O)(O)OC3OC(CO)C(O)C(O)C3O)C(O)C2O)c(=O)[nH]1', 'OCC(O)CO', 'CC(C)CCCC(C)C1CCC2C3CC=C4CC(O)CCC4(C)C3CCC12C', 'CCCCCCCCCCCCCCCCCC(=O)[O-]']
2702
57
{'r_0120': ['CCCCCCCC=CC(=O)SCCNC(=O)CCNC(=O)C(O)C(C)(C)COP(=O)(O)OP(=O)(O)OCC1OC(n2cnc3c(N)ncnc32)C(O)C1OP(=O)(O)O', 'CCCCCCCCCC(=O)SCCNC(=O)CCNC(=O)C(O)C(C)(C)COP(=O)(O)OP(=O)(O)OCC1OC(n2cnc3c(N)ncnc32)C(O)C1OP(=O)(O)O'], 'r_0121': ['CCCCCCCCCC=CC(=O)SCCNC(=O)CCNC(=O)C(O)C(C)(C)COP(=O)(O)OP(=O)(O)OCC1OC(n2cnc3c(N)ncnc32)C(O)C1OP(=O)(O)O', 'CCCCCCCCCCCC(=O)SCCNC(=O)CCNC(=O)C(O)C(C)(C)COP(=O)(O)OP(=O)(O)OCC1OC(n2cnc3c(N)ncnc32)C(O)C1OP(=O)(O)O'], 'r_0123': ['CCCCCCCCCCCCCCCC(

In [34]:
yeast8_reaction_in_rxndb = get_yeast8_reaction_in_rxndb_parallel(rxndb_total_info_to_model_path,reaID_with_smiles_dict,model_excel,miss_met_id,num_process=num_process)


(1252, 8)
(0, 8)


In [35]:
with open(yeast8_reaction_in_rxndb_json, 'w') as json_file:
    json.dump(yeast8_reaction_in_rxndb, json_file,indent=4)